In [1]:
import pandas as pd
import numpy as np
import os
import glob
import datetime

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
import tensorflow

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, Input
from sklearn import preprocessing

import tensorflow as tf
import tensorflow

import numpy as np
import os
import pandas as pd
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import datetime
from os.path import isfile, join
from sys import getsizeof
import glob

from random import *

import matplotlib.pyplot as plt

import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder



Using all the data, using a previous model to predict the auroral regions

In [2]:

file_load_df_cumulative = '../ParticlePrecipitation/ML_DB_subsamp_ext_full_dfCumulative_complexHemisphereCombine.csv'
DMSP_DATA_DIR=''
df_cumulative = pd.read_csv(os.path.join(DMSP_DATA_DIR,file_load_df_cumulative))
df_cumulative = df_cumulative.sort_values(by=['ID_SC', 'Datetimes'])
df_cumulative = df_cumulative.set_index('Datetimes')
df_cumulative.index = pd.to_datetime(df_cumulative.index)

cols_to_drop_validation = [c for c in df_cumulative.columns if ('STD' in c) | ('AVG' in c) | ('SC_AACGM_LTIME'==c)]
# cols_to_drop_validation = [c for c in df.columns if ('1min' in c) | ('3min' in c) | ('4min' in c) | ('5min' in c) | ('15min' in c) | ('newell' in c) | ('STD' in c) | ('AVG' in c) | ('SC_AACGM_LTIME'==c)]

df_cumulative = df_cumulative.drop(columns=cols_to_drop_validation)

# Separate training and testing data
mask_val = [(df_cumulative.index.year == 2010) & (df_cumulative['ID_SC'].values==16)]
df_val = df_cumulative[mask_val[0]].copy(deep=True)
df_train = df_cumulative.copy(deep=True).drop( df_cumulative.index[mask_val[0]])

# mask_for_2010_to_2014 = [(df_train.index.year == 2010) | (df_train.index.year == 2011)
#                          | (df_train.index.year == 2012) |(df_train.index.year == 2013)
#                          | (df_train.index.year == 2014)]
# df_train = df_train[mask_for_2010_to_2014[0]]
df_train = df_train.sort_values(by=['ID_SC', 'Datetimes'])
df_val = df_val.sort_values(by=['ID_SC', 'Datetimes'])

# Construct X and y
feature_cols = [c for c in df_train.columns if not 'ELE' in c]
#print( (feature_cols))
#print(df_cumulative.columns)
X_val = df_val[feature_cols].copy(deep=True)
y_val = df_val['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
X_train = df_train[feature_cols].copy(deep=True)
y_train = df_train['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
scaler_X = preprocessing.RobustScaler()
scaler_X = scaler_X.fit(X_train.values)
X_val_scaled = scaler_X.transform(X_val.values)
X_train_scaled = scaler_X.transform(X_train.values)

numFeatures = len(X_train.columns.to_list())
feature_labels = X_train.columns.to_list()
y_train_erg = y_train.copy(deep=True) * (1.60218e-12)
y_val_erg = y_val.copy(deep=True) * (1.60218e-12)

y_train[y_train == 0] = 0.0001
y_val[y_val == 0] = 0.0001
y_train_log = np.log10(y_train.copy(deep=True))
y_val_log = np.log10(y_val.copy(deep=True))
X = np.array(X_train_scaled, dtype=np.float32)
X_test = np.array(X_val_scaled, dtype=np.float32)

In [3]:
from tensorflow.keras.layers import Input

input1 = Input(shape=(148))
model1 = Dense(64,  activation='relu')(input1)
model1 = Dense(32, activation='relu')(model1)
model1 = Dense(int(256),   activation='relu')(model1)
model1 = Dense(int(1024),  activation='relu')(model1)
model1 = Dense(int(1024),  activation='relu')(model1)
model1 = Dense(int(256),  activation='relu')(model1)
model1 = Dense(32, activation='relu')(model1)
model1 = Dense(4, activation='relu')(model1)
model1 = Dense(1, )(model1)


output = model1
model = tensorflow.keras.models.Model(inputs=input1, outputs=output)
#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam')
model.summary()



Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 148)]             0         
_________________________________________________________________
dense (Dense)                (None, 64)                9536      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               8448      
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              263168    
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_5 (Dense)              (None, 256)               262400

In [5]:
history = model.fit(X, 
                   y_train_log, 
                    validation_data=(
                        X_test, 
                        y_val_log,                                    ),
                    batch_size=32768,epochs=400,verbose=2,
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',  restore_best_weights=True,
             patience=50)])#

Epoch 1/400
57/57 - 5s - loss: 1.0818 - val_loss: 1.6155
Epoch 2/400
57/57 - 5s - loss: 0.9561 - val_loss: 1.3255
Epoch 3/400
57/57 - 5s - loss: 0.8910 - val_loss: 1.1849
Epoch 4/400
57/57 - 5s - loss: 0.7307 - val_loss: 1.1283
Epoch 5/400
57/57 - 5s - loss: 0.7022 - val_loss: 1.1607
Epoch 6/400
57/57 - 5s - loss: 0.6625 - val_loss: 1.1177
Epoch 7/400
57/57 - 5s - loss: 0.6407 - val_loss: 0.9688
Epoch 8/400
57/57 - 5s - loss: 0.6275 - val_loss: 1.0439
Epoch 9/400
57/57 - 5s - loss: 0.6021 - val_loss: 1.0733
Epoch 10/400
57/57 - 5s - loss: 0.6101 - val_loss: 1.0426
Epoch 11/400
57/57 - 5s - loss: 0.5762 - val_loss: 0.8915
Epoch 12/400
57/57 - 5s - loss: 0.5774 - val_loss: 0.8702
Epoch 13/400
57/57 - 5s - loss: 0.5724 - val_loss: 0.9481
Epoch 14/400
57/57 - 5s - loss: 0.5556 - val_loss: 0.9408
Epoch 15/400
57/57 - 5s - loss: 0.5572 - val_loss: 0.8525
Epoch 16/400
57/57 - 5s - loss: 0.5472 - val_loss: 0.8426
Epoch 17/400
57/57 - 5s - loss: 0.5529 - val_loss: 0.8552
Epoch 18/400
57/57 - 5s

Epoch 142/400
57/57 - 5s - loss: 0.3821 - val_loss: 0.6262
Epoch 143/400
57/57 - 5s - loss: 0.3695 - val_loss: 0.6182
Epoch 144/400
57/57 - 5s - loss: 0.3681 - val_loss: 0.6295
Epoch 145/400
57/57 - 5s - loss: 0.3749 - val_loss: 0.6213
Epoch 146/400
57/57 - 5s - loss: 0.3687 - val_loss: 0.6136
Epoch 147/400
57/57 - 5s - loss: 0.3684 - val_loss: 0.6226
Epoch 148/400
57/57 - 5s - loss: 0.3634 - val_loss: 0.6232
Epoch 149/400
57/57 - 5s - loss: 0.3627 - val_loss: 0.6241
Epoch 150/400
57/57 - 5s - loss: 0.3628 - val_loss: 0.6734
Epoch 151/400
57/57 - 5s - loss: 0.3699 - val_loss: 0.6126
Epoch 152/400
57/57 - 5s - loss: 0.3621 - val_loss: 0.6411
Epoch 153/400
57/57 - 5s - loss: 0.3685 - val_loss: 0.6296
Epoch 154/400
57/57 - 5s - loss: 0.3650 - val_loss: 0.6092
Epoch 155/400
57/57 - 5s - loss: 0.3569 - val_loss: 0.6179
Epoch 156/400
57/57 - 5s - loss: 0.3616 - val_loss: 0.6308
Epoch 157/400
57/57 - 5s - loss: 0.3622 - val_loss: 0.6252
Epoch 158/400
57/57 - 5s - loss: 0.3565 - val_loss: 0.61

In [6]:
model.save('all_years_flux_only_relu_nodropout_best_epoch.model')
model.save_weights('all_years_flux_only_relu_nodropout_best_epoch.weights')
model.load_weights('all_years_flux_only_relu_nodropout_best_epoch.weights')

W0909 16:35:36.713745 139846239283008 deprecation.py:506] From /home/jackalak/.local/lib/python3.6/site-packages/tensorflow/python/ops/resource_variable_ops.py:1817: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [27]:
from tensorflow.keras.layers import Input

input1 = Input(shape=(148))
model1 = Dense(64,  activation='relu')(input1)
model1 = Dense(32, activation='relu')(model1)
model1 = Dense(int(256),   activation='relu')(model1)
model1 = Dense(int(1024),  activation='relu')(model1)
model1 = Dense(int(1024),  activation='relu')(model1)
model1 = model1 = Dropout(0.5)(model1)
model1 = Dense(int(256),  activation='relu')(model1)
model1 = Dense(32, activation='relu')(model1)
model1 = Dense(4, activation='relu')(model1)
model1 = Dense(1, )(model1)


output = model1
model = tensorflow.keras.models.Model(inputs=input1, outputs=output)
#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam')
model.summary()



Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 148)]             0         
_________________________________________________________________
dense_63 (Dense)             (None, 64)                9536      
_________________________________________________________________
dense_64 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_65 (Dense)             (None, 256)               8448      
_________________________________________________________________
dense_66 (Dense)             (None, 1024)              263168    
_________________________________________________________________
dense_67 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dropout_8 (Dropout)          (None, 1024)              0   

In [28]:
history = model.fit(X, 
                   y_train_log, 
                    validation_data=(
                        X_test, 
                        y_val_log,                                    ),
                    batch_size=32768,epochs=1000,verbose=2,
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',  restore_best_weights=True,
             patience=100)])#

Epoch 1/1000
57/57 - 6s - loss: 12.8808 - val_loss: 2.4505
Epoch 2/1000
57/57 - 6s - loss: 1.7073 - val_loss: 2.0842
Epoch 3/1000
57/57 - 6s - loss: 1.4838 - val_loss: 1.9183
Epoch 4/1000
57/57 - 6s - loss: 1.3907 - val_loss: 1.8234
Epoch 5/1000
57/57 - 6s - loss: 1.3313 - val_loss: 1.7671
Epoch 6/1000
57/57 - 6s - loss: 1.2574 - val_loss: 1.5782
Epoch 7/1000
57/57 - 6s - loss: 1.1546 - val_loss: 1.3345
Epoch 8/1000
57/57 - 6s - loss: 1.0252 - val_loss: 1.2602
Epoch 9/1000
57/57 - 6s - loss: 0.9188 - val_loss: 1.2800
Epoch 10/1000
57/57 - 6s - loss: 0.8638 - val_loss: 1.3413
Epoch 11/1000
57/57 - 6s - loss: 0.8203 - val_loss: 1.1466
Epoch 12/1000
57/57 - 6s - loss: 0.8207 - val_loss: 1.2996
Epoch 13/1000
57/57 - 6s - loss: 0.7644 - val_loss: 1.2531
Epoch 14/1000
57/57 - 6s - loss: 0.7483 - val_loss: 1.0161
Epoch 15/1000
57/57 - 6s - loss: 0.7274 - val_loss: 1.5332
Epoch 16/1000
57/57 - 6s - loss: 0.7138 - val_loss: 1.0874
Epoch 17/1000
57/57 - 6s - loss: 0.6977 - val_loss: 1.0639
Epoch

Epoch 140/1000
57/57 - 6s - loss: 0.3595 - val_loss: 0.6142
Epoch 141/1000
57/57 - 6s - loss: 0.3588 - val_loss: 0.6196
Epoch 142/1000
57/57 - 6s - loss: 0.3581 - val_loss: 0.6071
Epoch 143/1000
57/57 - 6s - loss: 0.3560 - val_loss: 0.6027
Epoch 144/1000
57/57 - 6s - loss: 0.3558 - val_loss: 0.6201
Epoch 145/1000
57/57 - 6s - loss: 0.3555 - val_loss: 0.6017
Epoch 146/1000
57/57 - 5s - loss: 0.3536 - val_loss: 0.6024
Epoch 147/1000
57/57 - 5s - loss: 0.3523 - val_loss: 0.6107
Epoch 148/1000
57/57 - 6s - loss: 0.3517 - val_loss: 0.6032
Epoch 149/1000
57/57 - 6s - loss: 0.3515 - val_loss: 0.6139
Epoch 150/1000
57/57 - 6s - loss: 0.3500 - val_loss: 0.6065
Epoch 151/1000
57/57 - 6s - loss: 0.3499 - val_loss: 0.6446
Epoch 152/1000
57/57 - 6s - loss: 0.3504 - val_loss: 0.6220
Epoch 153/1000
57/57 - 6s - loss: 0.3474 - val_loss: 0.6232
Epoch 154/1000
57/57 - 6s - loss: 0.3486 - val_loss: 0.6221
Epoch 155/1000
57/57 - 6s - loss: 0.3469 - val_loss: 0.6163
Epoch 156/1000
57/57 - 5s - loss: 0.3454

In [29]:
model.save('all_years_flux_only_relu_dropout_best_epoch2.model')
model.save_weights('all_years_flux_only_relu_dropout_best_epoch2.weights')
model.load_weights('all_years_flux_only_relu_dropout_best_epoch2.weights')

In [36]:
from tensorflow.keras.layers import Input

input1 = Input(shape=(148))
model1 = Dense(256,  activation='relu')(input1)
model1 = model1 = Dropout(0.5)(model1)
model1 = Dense(64, activation='relu')(model1)
model1 = Dense(32, activation='relu')(model1)
model1 = Dense(int(256),   activation='relu')(model1)
model1 = Dense(int(1024),  activation='relu')(model1)
model1 = model1 = Dropout(0.2)(model1)
model1 = Dense(int(256),  activation='relu')(model1)
model1 = Dense(32, activation='relu')(model1)
model1 = Dense(4, activation='relu')(model1)
model1 = Dense(1, )(model1)


output = model1
model = tensorflow.keras.models.Model(inputs=input1, outputs=output)
#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam')
model.summary()

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 148)]             0         
_________________________________________________________________
dense_92 (Dense)             (None, 256)               38144     
_________________________________________________________________
dropout_13 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_93 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_94 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_95 (Dense)             (None, 256)               8448      
_________________________________________________________________
dense_96 (Dense)             (None, 1024)              263

In [37]:
history = model.fit(X, 
                   y_train_log, 
                    validation_data=(
                        X_test, 
                        y_val_log,                                    ),
                    batch_size=32768,epochs=1000,verbose=2,
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',  restore_best_weights=True,
             patience=100)])#

Epoch 1/1000
57/57 - 3s - loss: 18.7202 - val_loss: 3.0326
Epoch 2/1000
57/57 - 3s - loss: 2.5589 - val_loss: 2.2293
Epoch 3/1000
57/57 - 3s - loss: 1.5389 - val_loss: 2.1572
Epoch 4/1000
57/57 - 3s - loss: 1.4271 - val_loss: 2.0128
Epoch 5/1000
57/57 - 3s - loss: 1.3504 - val_loss: 1.8899
Epoch 6/1000
57/57 - 3s - loss: 1.2744 - val_loss: 1.6937
Epoch 7/1000
57/57 - 3s - loss: 1.2039 - val_loss: 1.5929
Epoch 8/1000
57/57 - 3s - loss: 1.1188 - val_loss: 1.4929
Epoch 9/1000
57/57 - 3s - loss: 1.0266 - val_loss: 1.2047
Epoch 10/1000
57/57 - 3s - loss: 0.9249 - val_loss: 1.0590
Epoch 11/1000
57/57 - 3s - loss: 0.8528 - val_loss: 1.0450
Epoch 12/1000
57/57 - 3s - loss: 0.8054 - val_loss: 1.0554
Epoch 13/1000
57/57 - 3s - loss: 0.7732 - val_loss: 0.9336
Epoch 14/1000
57/57 - 3s - loss: 0.7411 - val_loss: 0.8971
Epoch 15/1000
57/57 - 3s - loss: 0.7395 - val_loss: 0.8801
Epoch 16/1000
57/57 - 3s - loss: 0.7004 - val_loss: 0.8693
Epoch 17/1000
57/57 - 3s - loss: 0.6922 - val_loss: 0.8642
Epoch

Epoch 140/1000
57/57 - 3s - loss: 0.4512 - val_loss: 0.6633
Epoch 141/1000
57/57 - 3s - loss: 0.4512 - val_loss: 0.6345
Epoch 142/1000
57/57 - 3s - loss: 0.4502 - val_loss: 0.6731
Epoch 143/1000
57/57 - 3s - loss: 0.4525 - val_loss: 0.6426
Epoch 144/1000
57/57 - 3s - loss: 0.4489 - val_loss: 0.6593
Epoch 145/1000
57/57 - 3s - loss: 0.4490 - val_loss: 0.6741
Epoch 146/1000
57/57 - 3s - loss: 0.4484 - val_loss: 0.6499
Epoch 147/1000
57/57 - 3s - loss: 0.4519 - val_loss: 0.6299
Epoch 148/1000
57/57 - 3s - loss: 0.4486 - val_loss: 0.6641
Epoch 149/1000
57/57 - 3s - loss: 0.4517 - val_loss: 0.6377
Epoch 150/1000
57/57 - 3s - loss: 0.4464 - val_loss: 0.6327
Epoch 151/1000
57/57 - 3s - loss: 0.4474 - val_loss: 0.6558
Epoch 152/1000
57/57 - 3s - loss: 0.4474 - val_loss: 0.6251
Epoch 153/1000
57/57 - 3s - loss: 0.4459 - val_loss: 0.6205
Epoch 154/1000
57/57 - 3s - loss: 0.4477 - val_loss: 0.6360
Epoch 155/1000
57/57 - 3s - loss: 0.4473 - val_loss: 0.6391
Epoch 156/1000
57/57 - 3s - loss: 0.4455

Epoch 277/1000
57/57 - 3s - loss: 0.4228 - val_loss: 0.5776
Epoch 278/1000
57/57 - 3s - loss: 0.4231 - val_loss: 0.5797
Epoch 279/1000
57/57 - 3s - loss: 0.4233 - val_loss: 0.5945
Epoch 280/1000
57/57 - 3s - loss: 0.4226 - val_loss: 0.5716
Epoch 281/1000
57/57 - 3s - loss: 0.4225 - val_loss: 0.5845
Epoch 282/1000
57/57 - 3s - loss: 0.4215 - val_loss: 0.5759
Epoch 283/1000
57/57 - 3s - loss: 0.4218 - val_loss: 0.5733
Epoch 284/1000
57/57 - 3s - loss: 0.4223 - val_loss: 0.5741
Epoch 285/1000
57/57 - 3s - loss: 0.4221 - val_loss: 0.5836
Epoch 286/1000
57/57 - 3s - loss: 0.4230 - val_loss: 0.5779
Epoch 287/1000
57/57 - 3s - loss: 0.4219 - val_loss: 0.5809
Epoch 288/1000
57/57 - 3s - loss: 0.4213 - val_loss: 0.5889
Epoch 289/1000
57/57 - 3s - loss: 0.4223 - val_loss: 0.5848
Epoch 290/1000
57/57 - 3s - loss: 0.4221 - val_loss: 0.5689
Epoch 291/1000
57/57 - 3s - loss: 0.4221 - val_loss: 0.5818
Epoch 292/1000
57/57 - 3s - loss: 0.4207 - val_loss: 0.5914
Epoch 293/1000
57/57 - 3s - loss: 0.4210

Epoch 414/1000
57/57 - 3s - loss: 0.4112 - val_loss: 0.5696
Epoch 415/1000
57/57 - 3s - loss: 0.4107 - val_loss: 0.5828
Epoch 416/1000
57/57 - 3s - loss: 0.4109 - val_loss: 0.5750
Epoch 417/1000
57/57 - 3s - loss: 0.4110 - val_loss: 0.5778
Epoch 418/1000
57/57 - 3s - loss: 0.4104 - val_loss: 0.5752
Epoch 419/1000
57/57 - 3s - loss: 0.4109 - val_loss: 0.5857
Epoch 420/1000
57/57 - 3s - loss: 0.4105 - val_loss: 0.5743
Epoch 421/1000
57/57 - 3s - loss: 0.4111 - val_loss: 0.5715
Epoch 422/1000
57/57 - 3s - loss: 0.4109 - val_loss: 0.5736


In [38]:
model.save('all_years_flux_only_relu_dropout_best_epoch3.model')
model.save_weights('all_years_flux_only_relu_dropout_best_epoch3.weights')
model.load_weights('all_years_flux_only_relu_dropout_best_epoch3.weights')

In [20]:
from tensorflow.keras.layers import Input,Dropout

input1 = Input(shape=(148))
model1 = Dense(256,  activation='relu')(input1)
model1 = model1 = Dropout(0.5)(model1)
model1 = Dense(64,  activation='relu')(input1)
model1 = Dense(16, activation='relu')(model1)
model1 = Dense(4, activation='relu')(model1)
model1 = Dense(1, )(model1)


output = model1
model = tensorflow.keras.models.Model(inputs=input1, outputs=output)
#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam')
model.summary()



Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 148)]             0         
_________________________________________________________________
dense_50 (Dense)             (None, 64)                9536      
_________________________________________________________________
dense_51 (Dense)             (None, 16)                1040      
_________________________________________________________________
dense_52 (Dense)             (None, 4)                 68        
_________________________________________________________________
dense_53 (Dense)             (None, 1)                 5         
Total params: 10,649
Trainable params: 10,649
Non-trainable params: 0
_________________________________________________________________


In [21]:
history = model.fit(X, 
                   y_train_log, 
                    validation_data=(
                        X_test, 
                        y_val_log,                                    ),
                    batch_size=32768,epochs=600,verbose=2,
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',  restore_best_weights=True,
             patience=50)])#

Epoch 1/600
57/57 - 0s - loss: 75.4686 - val_loss: 52.6744
Epoch 2/600
57/57 - 0s - loss: 26.7343 - val_loss: 16.8137
Epoch 3/600
57/57 - 0s - loss: 11.0786 - val_loss: 6.8890
Epoch 4/600
57/57 - 0s - loss: 6.2690 - val_loss: 4.5887
Epoch 5/600
57/57 - 0s - loss: 4.4552 - val_loss: 3.7373
Epoch 6/600
57/57 - 0s - loss: 3.5842 - val_loss: 3.3024
Epoch 7/600
57/57 - 0s - loss: 3.0079 - val_loss: 3.0021
Epoch 8/600
57/57 - 0s - loss: 2.5947 - val_loss: 2.7788
Epoch 9/600
57/57 - 0s - loss: 2.2884 - val_loss: 2.5808
Epoch 10/600
57/57 - 0s - loss: 2.0534 - val_loss: 2.4426
Epoch 11/600
57/57 - 0s - loss: 1.8631 - val_loss: 2.3174
Epoch 12/600
57/57 - 0s - loss: 1.7324 - val_loss: 2.1659
Epoch 13/600
57/57 - 0s - loss: 1.6148 - val_loss: 2.1133
Epoch 14/600
57/57 - 0s - loss: 1.5365 - val_loss: 2.0505
Epoch 15/600
57/57 - 0s - loss: 1.4717 - val_loss: 2.0018
Epoch 16/600
57/57 - 0s - loss: 1.4190 - val_loss: 1.9595
Epoch 17/600
57/57 - 0s - loss: 1.3842 - val_loss: 1.8942
Epoch 18/600
57/57

Epoch 142/600
57/57 - 0s - loss: 0.4999 - val_loss: 0.7854
Epoch 143/600
57/57 - 0s - loss: 0.4975 - val_loss: 0.7756
Epoch 144/600
57/57 - 0s - loss: 0.4978 - val_loss: 0.7781
Epoch 145/600
57/57 - 0s - loss: 0.4974 - val_loss: 0.7742
Epoch 146/600
57/57 - 0s - loss: 0.4966 - val_loss: 0.7728
Epoch 147/600
57/57 - 0s - loss: 0.4966 - val_loss: 0.7670
Epoch 148/600
57/57 - 0s - loss: 0.4966 - val_loss: 0.7679
Epoch 149/600
57/57 - 0s - loss: 0.4974 - val_loss: 0.7685
Epoch 150/600
57/57 - 0s - loss: 0.4957 - val_loss: 0.7658
Epoch 151/600
57/57 - 0s - loss: 0.4947 - val_loss: 0.7678
Epoch 152/600
57/57 - 0s - loss: 0.4946 - val_loss: 0.7682
Epoch 153/600
57/57 - 0s - loss: 0.4942 - val_loss: 0.7697
Epoch 154/600
57/57 - 0s - loss: 0.4957 - val_loss: 0.7649
Epoch 155/600
57/57 - 0s - loss: 0.4942 - val_loss: 0.7652
Epoch 156/600
57/57 - 0s - loss: 0.4933 - val_loss: 0.7634
Epoch 157/600
57/57 - 0s - loss: 0.4938 - val_loss: 0.7612
Epoch 158/600
57/57 - 0s - loss: 0.4924 - val_loss: 0.76

Epoch 281/600
57/57 - 0s - loss: 0.4747 - val_loss: 0.6753
Epoch 282/600
57/57 - 0s - loss: 0.4650 - val_loss: 0.6606
Epoch 283/600
57/57 - 0s - loss: 0.4564 - val_loss: 0.6652
Epoch 284/600
57/57 - 0s - loss: 0.4545 - val_loss: 0.6573
Epoch 285/600
57/57 - 0s - loss: 0.4536 - val_loss: 0.6599
Epoch 286/600
57/57 - 0s - loss: 0.4530 - val_loss: 0.6589
Epoch 287/600
57/57 - 0s - loss: 0.4527 - val_loss: 0.6625
Epoch 288/600
57/57 - 0s - loss: 0.4524 - val_loss: 0.6573
Epoch 289/600
57/57 - 0s - loss: 0.4525 - val_loss: 0.6656
Epoch 290/600
57/57 - 0s - loss: 0.4542 - val_loss: 0.6633
Epoch 291/600
57/57 - 0s - loss: 0.4517 - val_loss: 0.6576
Epoch 292/600
57/57 - 0s - loss: 0.4513 - val_loss: 0.6680
Epoch 293/600
57/57 - 0s - loss: 0.4519 - val_loss: 0.6618
Epoch 294/600
57/57 - 0s - loss: 0.4514 - val_loss: 0.6547
Epoch 295/600
57/57 - 0s - loss: 0.4513 - val_loss: 0.6611
Epoch 296/600
57/57 - 0s - loss: 0.4505 - val_loss: 0.6576
Epoch 297/600
57/57 - 0s - loss: 0.4504 - val_loss: 0.65

Epoch 420/600
57/57 - 0s - loss: 0.4425 - val_loss: 0.6365
Epoch 421/600
57/57 - 0s - loss: 0.4423 - val_loss: 0.6403
Epoch 422/600
57/57 - 0s - loss: 0.4410 - val_loss: 0.6391
Epoch 423/600
57/57 - 0s - loss: 0.4406 - val_loss: 0.6344
Epoch 424/600
57/57 - 0s - loss: 0.4399 - val_loss: 0.6422
Epoch 425/600
57/57 - 0s - loss: 0.4393 - val_loss: 0.6360
Epoch 426/600
57/57 - 0s - loss: 0.4395 - val_loss: 0.6451
Epoch 427/600
57/57 - 0s - loss: 0.4396 - val_loss: 0.6444
Epoch 428/600
57/57 - 0s - loss: 0.4388 - val_loss: 0.6403
Epoch 429/600
57/57 - 0s - loss: 0.4391 - val_loss: 0.6425
Epoch 430/600
57/57 - 0s - loss: 0.4391 - val_loss: 0.6435
Epoch 431/600
57/57 - 0s - loss: 0.4392 - val_loss: 0.6347
Epoch 432/600
57/57 - 0s - loss: 0.4388 - val_loss: 0.6390
Epoch 433/600
57/57 - 0s - loss: 0.4383 - val_loss: 0.6474
Epoch 434/600
57/57 - 0s - loss: 0.4384 - val_loss: 0.6450
Epoch 435/600
57/57 - 0s - loss: 0.4384 - val_loss: 0.6369
Epoch 436/600
57/57 - 0s - loss: 0.4390 - val_loss: 0.64

Epoch 559/600
57/57 - 0s - loss: 0.4324 - val_loss: 0.6312
Epoch 560/600
57/57 - 0s - loss: 0.4321 - val_loss: 0.6386
Epoch 561/600
57/57 - 0s - loss: 0.4314 - val_loss: 0.6331
Epoch 562/600
57/57 - 0s - loss: 0.4317 - val_loss: 0.6282
Epoch 563/600
57/57 - 0s - loss: 0.4316 - val_loss: 0.6354
Epoch 564/600
57/57 - 0s - loss: 0.4342 - val_loss: 0.6343
Epoch 565/600
57/57 - 0s - loss: 0.4346 - val_loss: 0.6307
Epoch 566/600
57/57 - 0s - loss: 0.4331 - val_loss: 0.6277
Epoch 567/600
57/57 - 0s - loss: 0.4317 - val_loss: 0.6287
Epoch 568/600
57/57 - 0s - loss: 0.4320 - val_loss: 0.6304
Epoch 569/600
57/57 - 0s - loss: 0.4319 - val_loss: 0.6295
Epoch 570/600
57/57 - 0s - loss: 0.4315 - val_loss: 0.6255
Epoch 571/600
57/57 - 0s - loss: 0.4312 - val_loss: 0.6327
Epoch 572/600
57/57 - 0s - loss: 0.4314 - val_loss: 0.6362
Epoch 573/600
57/57 - 0s - loss: 0.4316 - val_loss: 0.6291
Epoch 574/600
57/57 - 0s - loss: 0.4317 - val_loss: 0.6278
Epoch 575/600
57/57 - 0s - loss: 0.4318 - val_loss: 0.62

In [24]:
history = model.fit(X, 
                   y_train_log, 
                    validation_data=(
                        X_test, 
                        y_val_log,                                    ),
                    batch_size=32768,epochs=600,verbose=2,
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',  restore_best_weights=True,
             patience=50)])#

Epoch 1/600
57/57 - 0s - loss: 0.4311 - val_loss: 0.6284
Epoch 2/600
57/57 - 0s - loss: 0.4312 - val_loss: 0.6276
Epoch 3/600
57/57 - 0s - loss: 0.4335 - val_loss: 0.6290
Epoch 4/600
57/57 - 0s - loss: 0.4312 - val_loss: 0.6297
Epoch 5/600
57/57 - 0s - loss: 0.4309 - val_loss: 0.6262
Epoch 6/600
57/57 - 0s - loss: 0.4313 - val_loss: 0.6302
Epoch 7/600
57/57 - 0s - loss: 0.4309 - val_loss: 0.6292
Epoch 8/600
57/57 - 0s - loss: 0.4313 - val_loss: 0.6239
Epoch 9/600
57/57 - 0s - loss: 0.4318 - val_loss: 0.6254
Epoch 10/600
57/57 - 0s - loss: 0.4307 - val_loss: 0.6256
Epoch 11/600
57/57 - 0s - loss: 0.4314 - val_loss: 0.6246
Epoch 12/600
57/57 - 0s - loss: 0.4313 - val_loss: 0.6204
Epoch 13/600
57/57 - 0s - loss: 0.4316 - val_loss: 0.6248
Epoch 14/600
57/57 - 0s - loss: 0.4306 - val_loss: 0.6316
Epoch 15/600
57/57 - 0s - loss: 0.4314 - val_loss: 0.6209
Epoch 16/600
57/57 - 0s - loss: 0.4305 - val_loss: 0.6252
Epoch 17/600
57/57 - 0s - loss: 0.4302 - val_loss: 0.6263
Epoch 18/600
57/57 - 0s

Epoch 142/600
57/57 - 1s - loss: 0.4268 - val_loss: 0.6148
Epoch 143/600
57/57 - 1s - loss: 0.4275 - val_loss: 0.6125
Epoch 144/600
57/57 - 0s - loss: 0.4272 - val_loss: 0.6137
Epoch 145/600
57/57 - 0s - loss: 0.4280 - val_loss: 0.6104
Epoch 146/600
57/57 - 0s - loss: 0.4269 - val_loss: 0.6112
Epoch 147/600
57/57 - 0s - loss: 0.4270 - val_loss: 0.6118
Epoch 148/600
57/57 - 1s - loss: 0.4271 - val_loss: 0.6103
Epoch 149/600
57/57 - 0s - loss: 0.4271 - val_loss: 0.6120
Epoch 150/600
57/57 - 0s - loss: 0.4273 - val_loss: 0.6160
Epoch 151/600
57/57 - 0s - loss: 0.4272 - val_loss: 0.6118
Epoch 152/600
57/57 - 0s - loss: 0.4266 - val_loss: 0.6117
Epoch 153/600
57/57 - 0s - loss: 0.4268 - val_loss: 0.6108
Epoch 154/600
57/57 - 0s - loss: 0.4266 - val_loss: 0.6190
Epoch 155/600
57/57 - 0s - loss: 0.4264 - val_loss: 0.6161
Epoch 156/600
57/57 - 0s - loss: 0.4265 - val_loss: 0.6123
Epoch 157/600
57/57 - 0s - loss: 0.4268 - val_loss: 0.6117
Epoch 158/600
57/57 - 0s - loss: 0.4263 - val_loss: 0.61

Epoch 281/600
57/57 - 0s - loss: 0.4233 - val_loss: 0.6083
Epoch 282/600
57/57 - 0s - loss: 0.4230 - val_loss: 0.6097
Epoch 283/600
57/57 - 0s - loss: 0.4228 - val_loss: 0.6070
Epoch 284/600
57/57 - 0s - loss: 0.4234 - val_loss: 0.6083
Epoch 285/600
57/57 - 0s - loss: 0.4232 - val_loss: 0.6060
Epoch 286/600
57/57 - 0s - loss: 0.4230 - val_loss: 0.6079
Epoch 287/600
57/57 - 0s - loss: 0.4247 - val_loss: 0.6067
Epoch 288/600
57/57 - 0s - loss: 0.4234 - val_loss: 0.6081
Epoch 289/600
57/57 - 0s - loss: 0.4233 - val_loss: 0.6096
Epoch 290/600
57/57 - 0s - loss: 0.4230 - val_loss: 0.6104
Epoch 291/600
57/57 - 0s - loss: 0.4232 - val_loss: 0.6057
Epoch 292/600
57/57 - 0s - loss: 0.4235 - val_loss: 0.6081
Epoch 293/600
57/57 - 0s - loss: 0.4238 - val_loss: 0.6080
Epoch 294/600
57/57 - 0s - loss: 0.4236 - val_loss: 0.6068
Epoch 295/600
57/57 - 0s - loss: 0.4243 - val_loss: 0.6064
Epoch 296/600
57/57 - 0s - loss: 0.4245 - val_loss: 0.6117
Epoch 297/600
57/57 - 0s - loss: 0.4234 - val_loss: 0.60

Epoch 420/600
57/57 - 0s - loss: 0.4219 - val_loss: 0.6029
Epoch 421/600
57/57 - 0s - loss: 0.4218 - val_loss: 0.6155
Epoch 422/600
57/57 - 0s - loss: 0.4215 - val_loss: 0.6067
Epoch 423/600
57/57 - 0s - loss: 0.4232 - val_loss: 0.6113
Epoch 424/600
57/57 - 0s - loss: 0.4217 - val_loss: 0.6071
Epoch 425/600
57/57 - 0s - loss: 0.4317 - val_loss: 0.6081
Epoch 426/600
57/57 - 0s - loss: 0.4243 - val_loss: 0.6143
Epoch 427/600
57/57 - 0s - loss: 0.4229 - val_loss: 0.6067
Epoch 428/600
57/57 - 0s - loss: 0.4218 - val_loss: 0.6104
Epoch 429/600
57/57 - 0s - loss: 0.4221 - val_loss: 0.6067
Epoch 430/600
57/57 - 0s - loss: 0.4222 - val_loss: 0.6093
Epoch 431/600
57/57 - 0s - loss: 0.4217 - val_loss: 0.6111
Epoch 432/600
57/57 - 0s - loss: 0.4218 - val_loss: 0.6097
Epoch 433/600
57/57 - 0s - loss: 0.4220 - val_loss: 0.6105
Epoch 434/600
57/57 - 0s - loss: 0.4213 - val_loss: 0.6067
Epoch 435/600
57/57 - 0s - loss: 0.4214 - val_loss: 0.6134
Epoch 436/600
57/57 - 0s - loss: 0.4221 - val_loss: 0.60

In [23]:
model.save('all_years_flux_only_relu_dropout_best_epoch.model')
model.save_weights('all_years_flux_only_relu_dropout_best_epoch.weights')
model.load_weights('all_years_flux_only_relu_dropout_best_epoch.weights')

In [34]:
from tensorflow.keras.layers import Input,Dropout

input1 = Input(shape=(148))
model1 = Dense(256,  activation='relu')(input1)
model1 = model1 = Dropout(0.5)(model1)
model1 = Dense(64,  activation='relu')(input1)
model1 = Dense(16, activation='relu')(model1)
model1 = Dense(1, )(model1)


output = model1
model = tensorflow.keras.models.Model(inputs=input1, outputs=output)
#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam')
model.summary()

history = model.fit(X, 
                   y_train_log, 
                    validation_data=(
                        X_test, 
                        y_val_log,                                    ),
                    batch_size=8192,epochs=2000,verbose=2,
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',  restore_best_weights=True,
             patience=100)])#

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 148)]             0         
_________________________________________________________________
dense_89 (Dense)             (None, 64)                9536      
_________________________________________________________________
dense_90 (Dense)             (None, 16)                1040      
_________________________________________________________________
dense_91 (Dense)             (None, 1)                 17        
Total params: 10,593
Trainable params: 10,593
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2000
225/225 - 1s - loss: 13.7932 - val_loss: 3.5220
Epoch 2/2000
225/225 - 1s - loss: 2.8927 - val_loss: 2.5730
Epoch 3/2000
225/225 - 1s - loss: 1.8952 - val_loss: 2.1676
Epoch 4/2000
225/225 - 1s - loss: 1.5209 - val_loss: 1.9984
Epoch 5/200

Epoch 121/2000
225/225 - 1s - loss: 0.4419 - val_loss: 0.6528
Epoch 122/2000
225/225 - 1s - loss: 0.4420 - val_loss: 0.6573
Epoch 123/2000
225/225 - 1s - loss: 0.4414 - val_loss: 0.6584
Epoch 124/2000
225/225 - 1s - loss: 0.4420 - val_loss: 0.6534
Epoch 125/2000
225/225 - 1s - loss: 0.4415 - val_loss: 0.6508
Epoch 126/2000
225/225 - 1s - loss: 0.4425 - val_loss: 0.6524
Epoch 127/2000
225/225 - 1s - loss: 0.4406 - val_loss: 0.6619
Epoch 128/2000
225/225 - 1s - loss: 0.4408 - val_loss: 0.6531
Epoch 129/2000
225/225 - 1s - loss: 0.4405 - val_loss: 0.6519
Epoch 130/2000
225/225 - 1s - loss: 0.4404 - val_loss: 0.6495
Epoch 131/2000
225/225 - 1s - loss: 0.4401 - val_loss: 0.6471
Epoch 132/2000
225/225 - 1s - loss: 0.4404 - val_loss: 0.6497
Epoch 133/2000
225/225 - 1s - loss: 0.4403 - val_loss: 0.6496
Epoch 134/2000
225/225 - 1s - loss: 0.4395 - val_loss: 0.6518
Epoch 135/2000
225/225 - 1s - loss: 0.4395 - val_loss: 0.6570
Epoch 136/2000
225/225 - 1s - loss: 0.4397 - val_loss: 0.6509
Epoch 13

225/225 - 1s - loss: 0.4283 - val_loss: 0.6121
Epoch 254/2000
225/225 - 1s - loss: 0.4282 - val_loss: 0.6146
Epoch 255/2000
225/225 - 1s - loss: 0.4277 - val_loss: 0.6179
Epoch 256/2000
225/225 - 1s - loss: 0.4279 - val_loss: 0.6115
Epoch 257/2000
225/225 - 1s - loss: 0.4281 - val_loss: 0.6137
Epoch 258/2000
225/225 - 1s - loss: 0.4279 - val_loss: 0.6134
Epoch 259/2000
225/225 - 1s - loss: 0.4297 - val_loss: 0.6147
Epoch 260/2000
225/225 - 1s - loss: 0.4285 - val_loss: 0.6143
Epoch 261/2000
225/225 - 1s - loss: 0.4287 - val_loss: 0.6177
Epoch 262/2000
225/225 - 1s - loss: 0.4287 - val_loss: 0.6152
Epoch 263/2000
225/225 - 1s - loss: 0.4285 - val_loss: 0.6110
Epoch 264/2000
225/225 - 1s - loss: 0.4280 - val_loss: 0.6148
Epoch 265/2000
225/225 - 1s - loss: 0.4275 - val_loss: 0.6139
Epoch 266/2000
225/225 - 1s - loss: 0.4279 - val_loss: 0.6152
Epoch 267/2000
225/225 - 1s - loss: 0.4279 - val_loss: 0.6147
Epoch 268/2000
225/225 - 1s - loss: 0.4274 - val_loss: 0.6121
Epoch 269/2000
225/225 

Epoch 386/2000
225/225 - 1s - loss: 0.4233 - val_loss: 0.6060
Epoch 387/2000
225/225 - 1s - loss: 0.4241 - val_loss: 0.6061
Epoch 388/2000
225/225 - 1s - loss: 0.4234 - val_loss: 0.6089
Epoch 389/2000
225/225 - 1s - loss: 0.4234 - val_loss: 0.6115
Epoch 390/2000
225/225 - 1s - loss: 0.4237 - val_loss: 0.6039
Epoch 391/2000
225/225 - 1s - loss: 0.4238 - val_loss: 0.6077
Epoch 392/2000
225/225 - 1s - loss: 0.4233 - val_loss: 0.6036
Epoch 393/2000
225/225 - 1s - loss: 0.4231 - val_loss: 0.6077
Epoch 394/2000
225/225 - 1s - loss: 0.4232 - val_loss: 0.6051
Epoch 395/2000
225/225 - 1s - loss: 0.4235 - val_loss: 0.6068
Epoch 396/2000
225/225 - 1s - loss: 0.4233 - val_loss: 0.6061
Epoch 397/2000
225/225 - 1s - loss: 0.4233 - val_loss: 0.6045
Epoch 398/2000
225/225 - 1s - loss: 0.4232 - val_loss: 0.6060
Epoch 399/2000
225/225 - 1s - loss: 0.4233 - val_loss: 0.6079
Epoch 400/2000
225/225 - 1s - loss: 0.4230 - val_loss: 0.6078
Epoch 401/2000
225/225 - 1s - loss: 0.4230 - val_loss: 0.6082
Epoch 40

225/225 - 1s - loss: 0.4216 - val_loss: 0.6054
Epoch 519/2000
225/225 - 1s - loss: 0.4214 - val_loss: 0.6046
Epoch 520/2000
225/225 - 1s - loss: 0.4208 - val_loss: 0.6002
Epoch 521/2000
225/225 - 1s - loss: 0.4208 - val_loss: 0.6093
Epoch 522/2000
225/225 - 1s - loss: 0.4209 - val_loss: 0.6022
Epoch 523/2000
225/225 - 1s - loss: 0.4207 - val_loss: 0.6054
Epoch 524/2000
225/225 - 1s - loss: 0.4209 - val_loss: 0.6008
Epoch 525/2000
225/225 - 1s - loss: 0.4227 - val_loss: 0.6035
Epoch 526/2000
225/225 - 1s - loss: 0.4228 - val_loss: 0.6006
Epoch 527/2000
225/225 - 1s - loss: 0.4204 - val_loss: 0.6042
Epoch 528/2000
225/225 - 1s - loss: 0.4206 - val_loss: 0.6047
Epoch 529/2000
225/225 - 1s - loss: 0.4206 - val_loss: 0.5999
Epoch 530/2000
225/225 - 1s - loss: 0.4206 - val_loss: 0.6051
Epoch 531/2000
225/225 - 1s - loss: 0.4203 - val_loss: 0.6036
Epoch 532/2000
225/225 - 1s - loss: 0.4210 - val_loss: 0.5993
Epoch 533/2000
225/225 - 1s - loss: 0.4204 - val_loss: 0.6020
Epoch 534/2000
225/225 

Epoch 651/2000
225/225 - 1s - loss: 0.4188 - val_loss: 0.5988
Epoch 652/2000
225/225 - 1s - loss: 0.4193 - val_loss: 0.5986
Epoch 653/2000
225/225 - 1s - loss: 0.4189 - val_loss: 0.6004
Epoch 654/2000
225/225 - 1s - loss: 0.4195 - val_loss: 0.6000
Epoch 655/2000
225/225 - 1s - loss: 0.4190 - val_loss: 0.6009
Epoch 656/2000
225/225 - 1s - loss: 0.4191 - val_loss: 0.5998
Epoch 657/2000
225/225 - 1s - loss: 0.4191 - val_loss: 0.5992
Epoch 658/2000
225/225 - 1s - loss: 0.4191 - val_loss: 0.6007
Epoch 659/2000
225/225 - 1s - loss: 0.4190 - val_loss: 0.5970
Epoch 660/2000
225/225 - 1s - loss: 0.4192 - val_loss: 0.6003
Epoch 661/2000
225/225 - 1s - loss: 0.4191 - val_loss: 0.5984
Epoch 662/2000
225/225 - 1s - loss: 0.4188 - val_loss: 0.5992
Epoch 663/2000
225/225 - 1s - loss: 0.4184 - val_loss: 0.5992
Epoch 664/2000
225/225 - 1s - loss: 0.4187 - val_loss: 0.6000
Epoch 665/2000
225/225 - 1s - loss: 0.4188 - val_loss: 0.6038
Epoch 666/2000
225/225 - 1s - loss: 0.4186 - val_loss: 0.5976
Epoch 66

225/225 - 1s - loss: 0.4177 - val_loss: 0.5971
Epoch 784/2000
225/225 - 1s - loss: 0.4180 - val_loss: 0.5964
Epoch 785/2000
225/225 - 1s - loss: 0.4177 - val_loss: 0.5946
Epoch 786/2000
225/225 - 1s - loss: 0.4176 - val_loss: 0.5989
Epoch 787/2000
225/225 - 1s - loss: 0.4176 - val_loss: 0.5986
Epoch 788/2000
225/225 - 1s - loss: 0.4178 - val_loss: 0.5991
Epoch 789/2000
225/225 - 1s - loss: 0.4229 - val_loss: 0.5999
Epoch 790/2000
225/225 - 1s - loss: 0.4217 - val_loss: 0.5992
Epoch 791/2000
225/225 - 1s - loss: 0.4185 - val_loss: 0.5984
Epoch 792/2000
225/225 - 1s - loss: 0.4184 - val_loss: 0.5975
Epoch 793/2000
225/225 - 1s - loss: 0.4176 - val_loss: 0.6011
Epoch 794/2000
225/225 - 1s - loss: 0.4178 - val_loss: 0.5986
Epoch 795/2000
225/225 - 1s - loss: 0.4180 - val_loss: 0.5996
Epoch 796/2000
225/225 - 1s - loss: 0.4176 - val_loss: 0.5963
Epoch 797/2000
225/225 - 1s - loss: 0.4174 - val_loss: 0.5974
Epoch 798/2000
225/225 - 1s - loss: 0.4180 - val_loss: 0.5978
Epoch 799/2000
225/225 

In [35]:
model.save('all_years_flux_only_relu_dropout_best_epoch4.model')
model.save_weights('all_years_flux_only_relu_dropout_best_epoch4.weights')
model.load_weights('all_years_flux_only_relu_dropout_best_epoch4.weights')

In [39]:
from tensorflow.keras.layers import Input,Dropout

input1 = Input(shape=(148))
model1 = Dense(64,  activation='relu')(input1)
model1 = Dense(128,  activation='relu')(model1)
model1 = Dense(256,  activation='relu')(model1)

model1 = model1 = Dropout(0.5)(model1)
model1 = Dense(32, activation='relu')(model1)

model1 = Dense(4, activation='relu')(model1)
model1 = Dense(1, )(model1)


output = model1
model = tensorflow.keras.models.Model(inputs=input1, outputs=output)
#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam')
model.summary()

history = model.fit(X, 
                   y_train_log, 
                    validation_data=(
                        X_test, 
                        y_val_log,                                    ),
                    batch_size=32768,epochs=2000,verbose=2,
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',  restore_best_weights=True,
             patience=100)])#

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 148)]             0         
_________________________________________________________________
dense_101 (Dense)            (None, 64)                9536      
_________________________________________________________________
dense_102 (Dense)            (None, 128)               8320      
_________________________________________________________________
dense_103 (Dense)            (None, 256)               33024     
_________________________________________________________________
dropout_15 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_104 (Dense)            (None, 32)                8224      
_________________________________________________________________
dense_105 (Dense)            (None, 4)                 132

Epoch 116/2000
57/57 - 1s - loss: 0.4493 - val_loss: 1.7768
Epoch 117/2000
57/57 - 1s - loss: 0.4475 - val_loss: 1.8187
Epoch 118/2000
57/57 - 1s - loss: 0.4470 - val_loss: 1.9013
Epoch 119/2000
57/57 - 1s - loss: 0.4456 - val_loss: 1.8038
Epoch 120/2000
57/57 - 1s - loss: 0.4444 - val_loss: 1.8396
Epoch 121/2000
57/57 - 1s - loss: 0.4428 - val_loss: 1.9278
Epoch 122/2000
57/57 - 1s - loss: 0.4426 - val_loss: 1.8076
Epoch 123/2000
57/57 - 1s - loss: 0.4445 - val_loss: 1.7841
Epoch 124/2000
57/57 - 1s - loss: 0.4426 - val_loss: 1.8958
Epoch 125/2000
57/57 - 1s - loss: 0.4422 - val_loss: 1.7798
Epoch 126/2000
57/57 - 1s - loss: 0.4403 - val_loss: 1.8617
Epoch 127/2000
57/57 - 1s - loss: 0.4403 - val_loss: 1.6735
Epoch 128/2000
57/57 - 1s - loss: 0.4437 - val_loss: 1.8017


In [40]:
model.save('all_years_flux_only_relu_dropout_best_epoch5.model')
model.save_weights('all_years_flux_only_relu_dropout_best_epoch5.weights')
model.load_weights('all_years_flux_only_relu_dropout_best_epoch5.weights')

In [ ]:
from tensorflow.keras.layers import Input,Dropout

input1 = Input(shape=(148))
model1 = Dense(256,  activation='relu')(input1)
model1 = model1 = Dropout(0.5)(model1)
model1 = Dense(64,  activation='relu')(input1)
model1 = Dense(16, activation='relu')(model1)
model1 = Dense(1, )(model1)


output = model1
model = tensorflow.keras.models.Model(inputs=input1, outputs=output)
#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam')
model.summary()

history = model.fit(X, 
                   y_train_log, 
                    validation_data=(
                        X_test, 
                        y_val_log,                                    ),
                    batch_size=1024,epochs=1000,verbose=2,
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',  restore_best_weights=True,
             patience=50)])#

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 148)]             0         
_________________________________________________________________
dense_108 (Dense)            (None, 64)                9536      
_________________________________________________________________
dense_109 (Dense)            (None, 16)                1040      
_________________________________________________________________
dense_110 (Dense)            (None, 1)                 17        
Total params: 10,593
Trainable params: 10,593
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
1796/1796 - 3s - loss: 4.5481 - val_loss: 1.8216
Epoch 2/1000
1796/1796 - 3s - loss: 1.1298 - val_loss: 1.4881
Epoch 3/1000
1796/1796 - 3s - loss: 0.8258 - val_loss: 0.9754
Epoch 4/1000
1796/1796 - 2s - loss: 0.5865 - val_loss: 0.8790
Epoc

Epoch 117/1000
1796/1796 - 2s - loss: 0.4264 - val_loss: 0.6166
Epoch 118/1000
1796/1796 - 2s - loss: 0.4265 - val_loss: 0.6173
Epoch 119/1000
1796/1796 - 2s - loss: 0.4261 - val_loss: 0.6129
Epoch 120/1000
1796/1796 - 3s - loss: 0.4260 - val_loss: 0.6156
Epoch 121/1000
1796/1796 - 3s - loss: 0.4261 - val_loss: 0.6173
Epoch 122/1000
1796/1796 - 3s - loss: 0.4259 - val_loss: 0.6114
Epoch 123/1000
1796/1796 - 3s - loss: 0.4269 - val_loss: 0.6112
Epoch 124/1000
1796/1796 - 3s - loss: 0.4256 - val_loss: 0.6107
Epoch 125/1000
1796/1796 - 3s - loss: 0.4258 - val_loss: 0.6111
Epoch 126/1000
1796/1796 - 3s - loss: 0.4257 - val_loss: 0.6099
Epoch 127/1000
1796/1796 - 3s - loss: 0.4257 - val_loss: 0.6182
Epoch 128/1000
1796/1796 - 3s - loss: 0.4262 - val_loss: 0.6068
Epoch 129/1000
1796/1796 - 3s - loss: 0.4254 - val_loss: 0.6106
Epoch 130/1000
1796/1796 - 3s - loss: 0.4266 - val_loss: 0.6200
Epoch 131/1000
1796/1796 - 3s - loss: 0.4256 - val_loss: 0.6075
Epoch 132/1000
1796/1796 - 3s - loss: 0.

In [ ]:
model.save('all_years_flux_only_relu_dropout_best_epoch6.model')
model.save_weights('all_years_flux_only_relu_dropout_best_epoch6.weights')
model.load_weights('all_years_flux_only_relu_dropout_best_epoch6.weights')

In [ ]:
from tensorflow.keras.layers import Input

input1 = Input(shape=(148))
model1 = Dense(64,  activation='relu')(input1)
model1 = Dense(32, activation='relu')(model1)
model1 = Dense(int(256),   activation='relu')(model1)
model1 = Dense(int(1024),  activation='relu')(model1)
model1 = Dense(int(1024),  activation='relu')(model1)
model1 = model1 = Dropout(0.5)(model1)
model1 = Dense(int(256),  activation='relu')(model1)
model1 = Dense(32, activation='relu')(model1)
model1 = Dense(4, activation='relu')(model1)
model1 = Dense(1, )(model1)


output = model1
model = tensorflow.keras.models.Model(inputs=input1, outputs=output)
#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam')
model.summary()

history = model.fit(X, 
                   y_train_log, 
                    validation_data=(
                        X_test, 
                        y_val_log,                                    ),
                    batch_size=1024,epochs=1000,verbose=2,
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',  restore_best_weights=True,
             patience=50)])#

model.save('all_years_flux_only_relu_dropout_best_epoch6.model')
model.save_weights('all_years_flux_only_relu_dropout_best_epoch7.weights')
model.load_weights('all_years_flux_only_relu_dropout_best_epoch7.weights')